# Simulating Decoupling Protocols

In this notebook we first perform a series of simulated quantum noise spectroscopy experiment. The single-qubit semi-classical noisy Hamiltonian model is

$$H(t) = \sum_{i=x,y,z}\eta_i\sigma_i+\Omega(t)(\cos(\phi(t))\sigma_x+\sin(\phi(t))\sigma_y)\,.$$


We simulate this via a discrete SchWARMA model with discrete time Hamiltonian

$$H_k = \sum_{\ell=x,y,z}y^{(\ell)}_k\sigma_\ell+\Omega_k(\cos(phi_k)\sigma_x+\sin(\phi_k)\sigma_y)\,.$$


Additionally, we also simulate amplitude damping (T1) processes driven via SchWARMA models.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as pl
from mezze.random.SchWARMA import correlated_error #Helper function for squared exponential PSD driven models
import mezze.channel as ch

In [ ]:
#All noise will have average fixed average fidelity at first time step (replaces amplitude of underyling squared exponential spectrum)
fid = .99

#correlation time 
corr_time = 3
num_gates = 100
num_MC = 10000

#doing stochastic coherent z rotations, multi-axis coherent rotations, and damping
noise_types=['zrot','xyzrot','amp_damping']

#White and correlated with squared exponential spectrum
noise_corr={'white':1e-9,'corr':corr_time}

#Control schemes (defined below)
ctrl=['none','dd','xy4','pfr']

In [ ]:
# Define Pauli gates in QuantumChannel class
I = ch.QuantumChannel(ch._sigmaI,'unitary')
X = ch.QuantumChannel(ch._sigmaX,'unitary')
Y = ch.QuantumChannel(ch._sigmaY,'unitary')
Z = ch.QuantumChannel(ch._sigmaZ,'unitary')

# Defines simple control sequences b
# key: none - no control, identities only
#      dd - repeated X gates at each time step
#      xy4 - alternate between X and Y gates at each time step
#      pfr - pauli frame randomization, random X,Y,Z at each step (not used in paper)
def ctrl_gate(key,gate_num):
    if key=='none':
        return I
    elif key == 'dd':
        return X
    elif key == 'xy4':
        if gate_num % 2==0:
            return X
        else:
            return Y
    elif key == 'pfr':
        p = np.random.rand()
        if p<0:
            return I
        elif p<1./3:
            return X
        elif p<2./3:
            return Y
        else:
            return Z
    

In [ ]:
#Initialize all the various data structures so that they can be accesses as dictionaries

schwarmas ={}
fids = {}
unitality={}
ptms = {}
for key in noise_types:
    schwarmas[key]={}
    fids[key]={}
    unitality[key]={}
    ptms[key]={}
    for key2 in noise_corr.keys():
        schwarmas[key][key2]=correlated_error(key,noise_corr[key2],fid)
        fids[key][key2]={key3:np.ones((num_MC,num_gates+1))for key3 in ctrl}
        unitality[key][key2]={key3:np.ones((num_MC,num_gates+1))for key3 in ctrl}
        ptms[key][key2]={key3:[] for key3 in ctrl}

In [ ]:
#Actually run the Monte Carlo simulations and compute the various metrics
for mc in range(num_MC):
    for key in noise_types:
        for key2 in noise_corr.keys():
            for key3 in ctrl:
                prop = ch.QuantumChannel(np.eye(4),'liou')
                ideal = ch.QuantumChannel(np.eye(4),'liou')
                errs = schwarmas[key][key2].error_stream(num_gates)
                ptms[key][key2][key3].append([prop.ptm()])
                for lcv, gate in enumerate(errs):
                    ctrl_g = ctrl_gate(key3,lcv)
                    prop = gate*ctrl_g*prop
                    ideal = ctrl_g*ideal
                    fids[key][key2][key3][mc,lcv+1] = np.real(np.trace(prop.choi().dot(ideal.choi())))/4
                    unitality[key][key2][key3][mc,lcv+1]=1.0-np.linalg.norm(np.real(prop.ptm()[1:,0]))
                    ptms[key][key2][key3][-1].append(prop.ptm())

In [ ]:
pl.figure()
pl.plot(np.mean(fids['xyzrot']['white']['none'],0),'k',label='Mark. Depol.')
pl.plot(np.mean(fids['xyzrot']['white']['dd'],0),'-xk',markevery=5,label='XX Mark. Depol.')
pl.plot(np.mean(fids['xyzrot']['white']['xy4'],0),'-ok',markevery=7,label='XY4 Mark. Depol.')
pl.plot(np.mean(fids['xyzrot']['corr']['none'],0), '--r', label='Corr. Depol.')
pl.plot(np.mean(fids['xyzrot']['corr']['dd'],0), '--xr', markevery=5,label='XX Corr. Depol.')
pl.plot(np.mean(fids['xyzrot']['corr']['xy4'],0) ,'--or', markevery=7,label='XY4 Corr. Depol')
pl.ylabel('Process Fidelity',fontsize=14)
pl.xlabel('Time, in Gate Lengths',fontsize=14)
pl.title('Decoupling Multi-Axis Coherent Noise',fontsize=16)
pl.legend()
#pl.savefig('schwarma_ctrl_depol.eps',bbox_inches='tight')
pl.show()

In [ ]:
pl.figure()
pl.plot(np.mean(unitality['amp_damping']['white']['none'],0),'k',label='Mark. Damp')
pl.plot(np.mean(unitality['amp_damping']['corr']['none'],0),'--r',label='Corr. Damp')
pl.plot(np.mean(unitality['amp_damping']['white']['dd'],0),'-xk',markevery=5,label='XX Mark. Damp')
pl.plot(np.mean(unitality['amp_damping']['corr']['dd'],0),'--xr',markevery=7,label='XX Corr. Damp')
    
pl.legend()
pl.ylabel(r'Unitality, 1-$||\beta||$',fontsize=14)
pl.xlabel('Time, in Gate Lengths',fontsize=14)
pl.title('Decoupling Amplitude Damping',fontsize=16)
#pl.savefig('schwarma_ctrl_damp.eps',bbox_inches='tight')
pl.show()